Masked language modeling predicts a masked token in a sequence, and the model can attend to tokens bidirectionally. This means the model has full access to the tokens on the left and right. Masked language modeling is great for tasks that require a good contextual understanding of an entire sequence. BERT is an example of a masked language model.

In this notebook we will:
>Finetune DistilRoBERTa on the r/askscience subset of the ELI5 dataset.


>Use our finetuned model for inference.

We install all the necessary libraries installed:

In [1]:
! pip install transformers datasets evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.1 MB/s eta 0:00:00


In [2]:
!pip install --upgrade datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 15.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; pla

 **Load ELI5 dataset**

We start by  the first 5000 examples from the ELI5-Category dataset with the HuggingFace Datasets library.

In [3]:
from datasets import load_dataset

eli5 = load_dataset("eli5_category", split="train[:5000]")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/12.6k [00:00<?, ?B/s]

eli5_category.py:   0%|          | 0.00/4.17k [00:00<?, ?B/s]

The repository for eli5_category contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/eli5_category.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating train split:   0%|          | 0/91772 [00:00<?, ? examples/s]

Generating validation1 split:   0%|          | 0/5446 [00:00<?, ? examples/s]

Generating validation2 split:   0%|          | 0/2375 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5411 [00:00<?, ? examples/s]

We split the dataset's train split into a train and test set with the **train_test_split** method

In [4]:
eli5 = eli5.train_test_split(test_size=0.2)

Then we take a look at an example:

In [5]:
eli5["train"][0]

{'q_id': '7fru16',
 'title': 'Why are some cheeses made into wheels but others in blocks?',
 'selftext': '',
 'category': 'Other',
 'subreddit': 'explainlikeimfive',
 'answers': {'a_id': ['dqe5i81'],
  'text': ["Generally aged cheese is wheel shaped(think parmesan reggiano) and fresh cheese (like mozzarella) is in a block/sphere form because those shapes are more beneficial to their manufacture. Parmesan is shaped like that because each individual unit is big and heavy, so to make moving them around easier, they are shaped like a wheel so they could be rolled and pivoted around on their side. This is much easier than if they were in a block shape. The reason why they make the wheels so big and heavy is that the cheese ages better when it's that size. The wheel shape also gives flat sides to stably set the cheese down, while also allowing for better airflow than a box shape. Commercially made Mozzarella is shaped in blocks, because it doesn't have to age, so you can make the individual 

Preprocess

Form masked language modeling, the next step is to load a DistilBERTa tokenizer to process the text subfield:

In [6]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert/distilroberta-base")

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

We notice from the example above, the text field is actually nested inside answers. This means we need to extract the text subfield from its nested structure with the **flatten** method:

In [7]:
eli5 = eli5.flatten()
eli5["train"][0]

{'q_id': '7fru16',
 'title': 'Why are some cheeses made into wheels but others in blocks?',
 'selftext': '',
 'category': 'Other',
 'subreddit': 'explainlikeimfive',
 'answers.a_id': ['dqe5i81'],
 'answers.text': ["Generally aged cheese is wheel shaped(think parmesan reggiano) and fresh cheese (like mozzarella) is in a block/sphere form because those shapes are more beneficial to their manufacture. Parmesan is shaped like that because each individual unit is big and heavy, so to make moving them around easier, they are shaped like a wheel so they could be rolled and pivoted around on their side. This is much easier than if they were in a block shape. The reason why they make the wheels so big and heavy is that the cheese ages better when it's that size. The wheel shape also gives flat sides to stably set the cheese down, while also allowing for better airflow than a box shape. Commercially made Mozzarella is shaped in blocks, because it doesn't have to age, so you can make the individu

Each subfield is now a separate column as indicated by the answers prefix, and the text field is a list now. Instead of tokenizing each sentence separately, we convert the list to a string so you can jointly tokenize them.

Here is a first preprocessing function to join the list of strings for each example and tokenize the result:

In [8]:
def preprocess_function(examples):
    return tokenizer([" ".join(x) for x in examples["answers.text"]])

To apply this preprocessing function over the entire dataset,we use the HuggingFace Datasets map method.
 You speed up the map function by setting batched=True to process multiple elements of the dataset at once, and increasing the number of processes with num_proc.

In [9]:
tokenized_eli5 = eli5.map(
    preprocess_function,
    batched=True,
    num_proc=4,
    remove_columns=eli5["train"].column_names,
)

Map (num_proc=4):   0%|          | 0/4000 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (620 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1060 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (539 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1007 > 512). Running this sequence through the model will result in indexing errors


Map (num_proc=4):   0%|          | 0/1000 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (1560 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (682 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (4617 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (911 > 512). Running this sequence through the model will result in indexing errors


This dataset contains the token sequences, but some of these are longer than the maximum input length for the model.

We use a second preprocessing function to:
.Concatenate all the sequences
.Split the concatenated sequences into shorter chunks defined by block_size, which should be both shorter than the maximum input length and short enough for our GPU RAM.

In [10]:
block_size = 128


def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
    # customize this part to your needs.
    if total_length >= block_size:
        total_length = (total_length // block_size) * block_size
    # Split by chunks of block_size.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    return result

We apply the **group_texts** function over the entire dataset:

In [11]:
lm_dataset = tokenized_eli5.map(group_texts, batched=True, num_proc=4)

Map (num_proc=4):   0%|          | 0/4000 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/1000 [00:00<?, ? examples/s]

Now we create a batch of examples using DataCollatorForLanguageModeling. It’s more efficient to dynamically pad the sentences to the longest length in a batch during collation, instead of padding the whole dataset to the maximum length.

We use the end-of-sequence token as the padding token and specify **mlm_probability** to randomly mask tokens each time we iterate over the data

In [12]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15, return_tensors="tf")

**Train**

To finetune a model in TensorFlow,we start by setting up an optimizer function, learning rate schedule, and some training hyperparameters:

In [13]:
from transformers import create_optimizer, AdamWeightDecay

optimizer = AdamWeightDecay(learning_rate=2e-5, weight_decay_rate=0.01)

Then we can load DistilRoBERTa with TFAutoModelForMaskedLM:

In [14]:
from transformers import TFAutoModelForMaskedLM

model = TFAutoModelForMaskedLM.from_pretrained("distilbert/distilroberta-base")

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/331M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFRobertaForMaskedLM.

All the weights of TFRobertaForMaskedLM were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForMaskedLM for predictions without further training.


We convert our datasets to the **tf.data.Dataset** format with **prepare_tf_dataset**

In [15]:
tf_train_set = model.prepare_tf_dataset(
    lm_dataset["train"],
    shuffle=True,
    batch_size=16,
    collate_fn=data_collator,
)

tf_test_set = model.prepare_tf_dataset(
    lm_dataset["test"],
    shuffle=False,
    batch_size=16,
    collate_fn=data_collator,
)

We configure the model for training with **compile.**

In [16]:
import tensorflow as tf

model.compile(optimizer=optimizer)  # No loss argument!

Push to HubCallback

In [28]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) Y
Token is valid (permission: fineG

In [29]:
from transformers.keras_callbacks import PushToHubCallback

callback = PushToHubCallback(
    output_dir="my_awesome_eli5_mlm_model",
    tokenizer=tokenizer,
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'Repository' (from 'huggingface_hub.repository') is deprecated and will be removed from version '1.0'. Please prefer the http-based alternatives instead. Given its large adoption in legacy code, the complete removal is only planned on next major release.
For more details, please read https://huggingface.co/docs/huggingface_hub/concepts/git_vs_http.
  warnings.warn(warning_message, FutureWarning)
/content/my_awesome_eli5_mlm_model is already a clone of https://huggingface.co/Amine2003/my_awesome_eli5_mlm_model. Make sure you pull the latest changes with `repo.git_pull()`.


In [30]:
!git config --global user.email "mohammedamineelhajjam@gmail.com"
!git config --global user.name "Hajjamamine"



We call fit with our training and validation datasets, the number of epochs, and our callback to finetune the model:

In [31]:
model.fit(x=tf_train_set, validation_data=tf_test_set, epochs=3, callbacks=[callback])

Epoch 1/3
  5/659 [..............................] - ETA: 31s - loss: 2.1554

659/659 [==============================] - 51s 77ms/step - loss: 2.1285 - val_loss: 1.9907
Epoch 2/3
659/659 [==============================] - ETA: 0s - loss: 2.0967

Several commits (2) will be pushed upstream.


659/659 [==============================] - 50s 76ms/step - loss: 2.0967 - val_loss: 2.0160
Epoch 3/3
659/659 [==============================] - ETA: 0s - loss: 2.0846

Several commits (3) will be pushed upstream.


659/659 [==============================] - 51s 77ms/step - loss: 2.0846 - val_loss: 1.9929


 **Inference**

 We come up with some text you’d like the model to fill in the blank with, and use the special <mask> token to indicate the blank:

In [32]:
text = "The Milky Way is a <mask> galaxy."

We instantiate a pipeline for fill-mask with our model, and pass our text to it.

In [36]:
from transformers import pipeline

mask_filler = pipeline("fill-mask", "Amine2003/my_awesome_eli5_mlm_model")
mask_filler(text, top_k=3)

All model checkpoint layers were used when initializing TFRobertaForMaskedLM.

All the layers of TFRobertaForMaskedLM were initialized from the model checkpoint at Amine2003/my_awesome_eli5_mlm_model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForMaskedLM for predictions without further training.
Device set to use 0


[{'score': 0.3126121461391449,
  'token': 21300,
  'token_str': ' spiral',
  'sequence': 'The Milky Way is a spiral galaxy.'},
 {'score': 0.07564319670200348,
  'token': 2232,
  'token_str': ' massive',
  'sequence': 'The Milky Way is a massive galaxy.'},
 {'score': 0.062210116535425186,
  'token': 3065,
  'token_str': ' giant',
  'sequence': 'The Milky Way is a giant galaxy.'}]

We tokenize the text and return the **input_ids**

In [37]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("Amine2003/my_awesome_eli5_mlm_model")
inputs = tokenizer(text, return_tensors="tf")
mask_token_index = tf.where(inputs["input_ids"] == tokenizer.mask_token_id)[0, 1]

We pass your inputs to the model and return the logits of the masked token:

In [38]:
from transformers import TFAutoModelForMaskedLM

model = TFAutoModelForMaskedLM.from_pretrained("Amine2003/my_awesome_eli5_mlm_model")
logits = model(**inputs).logits
mask_token_logits = logits[0, mask_token_index, :]

All model checkpoint layers were used when initializing TFRobertaForMaskedLM.

All the layers of TFRobertaForMaskedLM were initialized from the model checkpoint at Amine2003/my_awesome_eli5_mlm_model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForMaskedLM for predictions without further training.


Then we return the three masked tokens with the highest proba and print them out:

In [39]:
top_3_tokens = tf.math.top_k(mask_token_logits, 3).indices.numpy()

for token in top_3_tokens:
    print(text.replace(tokenizer.mask_token, tokenizer.decode([token])))

The Milky Way is a  spiral galaxy.
The Milky Way is a  massive galaxy.
The Milky Way is a  giant galaxy.
